<a href="https://colab.research.google.com/github/kumuds4/BCH/blob/master/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making the Most of your Colab Subscription



## Faster GPUs

Users who have purchased one of Colab's paid plans have access to faster GPUs and more memory. You can upgrade your notebook's GPU settings in `Runtime > Change runtime type` in the menu to select from several accelerator options, subject to availability.

The free of charge version of Colab grants access to Nvidia's T4 GPUs subject to quota restrictions and availability.

You can see what GPU you've been assigned at any time by executing the following cell. If the execution result of running the code cell below is "Not connected to a GPU", you can change the runtime by going to `Runtime > Change runtime type` in the menu to enable a GPU accelerator, and then re-execute the code cell.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In order to use a GPU with your notebook, select the `Runtime > Change runtime type` menu, and then set the hardware accelerator to the desired option.

## More memory

Users who have purchased one of Colab's paid plans have access to high-memory VMs when they are available. More powerful GPUs are always offered with high-memory VMs.



You can see how much memory you have available at any time by running the following code cell. If the execution result of running the code cell below is "Not using a high-RAM runtime", then you can enable a high-RAM runtime via `Runtime > Change runtime type` in the menu. Then select High-RAM in the Runtime shape toggle button. After, re-execute the code cell.


In [ ]:
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Longer runtimes

All Colab runtimes are reset after some period of time (which is faster if the runtime isn't executing code). Colab Pro and Pro+ users have access to longer runtimes than those who use Colab free of charge.

## Background execution

Colab Pro+ users have access to background execution, where notebooks will continue executing even after you've closed a browser tab. This is always enabled in Pro+ runtimes as long as you have compute units available.



## Relaxing resource limits in Colab Pro

Your resources are not unlimited in Colab. To make the most of Colab, avoid using resources when you don't need them. For example, only use a GPU when required and close Colab tabs when finished.



If you encounter limitations, you can relax those limitations by purchasing more compute units via Pay As You Go. Anyone can purchase compute units via [Pay As You Go](https://colab.research.google.com/signup); no subscription is required.

## Send us feedback!

If you have any feedback for us, please let us know. The best way to send feedback is by using the Help > 'Send feedback...' menu. If you encounter usage limits in Colab Pro consider subscribing to Pro+.

If you encounter errors or other issues with billing (payments) for Colab Pro, Pro+, or Pay As You Go, please email [colab-billing@google.com](mailto:colab-billing@google.com).

## More Resources

### Working with Notebooks in Colab
- [Overview of Colab](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

<a name="working-with-data"></a>
### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb)
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas DataFrame](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Linear regression with tf.keras using synthetic data](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)


<a name="using-accelerated-hardware"></a>
### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TPUs in Colab](/notebooks/tpu.ipynb)

<a name="machine-learning-examples"></a>

## Machine Learning Examples

To see end-to-end examples of the interactive machine learning analyses that Colab makes possible, check out these tutorials using models from [TensorFlow Hub](https://tfhub.dev).

A few featured examples:

- [Retraining an Image Classifier](https://tensorflow.org/hub/tutorials/tf2_image_retraining): Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- [Text Classification](https://tensorflow.org/hub/tutorials/tf2_text_classification): Classify IMDB movie reviews as either *positive* or *negative*.
- [Style Transfer](https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization): Use deep learning to transfer style between images.
- [Multilingual Universal Sentence Encoder Q&A](https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa): Use a machine learning model to answer questions from the SQuAD dataset.
- [Video Interpolation](https://tensorflow.org/hub/tutorials/tweening_conv3d): Predict what happened in a video between the first and the last frame.


In [ ]:
# === Imports ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# ===========================
# Configuration Parameters
# ===========================
N = 128                # Block length
K = 64                 # Number of information bits
CRC_LEN = 8            # CRC length (bits)
LIST_SIZE = [1, 4, 8, 16]  # SCL list sizes
NUM_FRAMES_PER_SNR = 1000  # Number of frames per SNR point (reduce for quick tests)
SNR_RANGE_DB = np.arange(0.5, 4.5, 0.5)
DESIGN_SNR_DB = 0.5    # For frozen bit selection

BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 1e-3

# ===========================
# CRC Class (CRC-8)
# ===========================
class CRC:
    def __init__(self, poly=0x07, length=8):
        self.poly = poly
        self.length = length
        self.poly_mask = (1 << length)

    def compute(self, bits):
        reg = 0
        for bit in bits:
            reg = ((reg << 1) | bit) & ((1 << (self.length + 1)) - 1)
            if reg & self.poly_mask:
                reg ^= self.poly << (self.length - 1)
        crc = reg & ((1 << self.length) - 1)
        return crc

    def check(self, bits):
        # bits includes info + crc
        crc_calc = self.compute(bits[:-self.length])
        crc_received = 0
        for i in range(self.length):
            crc_received |= bits[-self.length + i] << (self.length - 1 - i)
        return crc_calc == crc_received

    def append_crc(self, bits):
        crc = self.compute(bits)
        crc_bits = [(crc >> (self.length - 1 - i)) & 1 for i in range(self.length)]
        return np.concatenate([bits, crc_bits])

# ===========================
# Polar Code Generator Class
# ===========================
class PolarCodeGenerator:
    def __init__(self, N, K, crc=None, design_snr_db=0.5):
        self.N = N
        self.K = K
        self.crc = crc
        self.design_snr_db = design_snr_db

        self.K_total = self.K + (self.crc.length if self.crc else 0)

        reliability_sequence = np.argsort(np.random.rand(N))
        self.frozen_bits = np.ones(N, dtype=bool)
        self.information_bits_positions = np.sort(reliability_sequence[:self.K_total])
        self.frozen_bits[self.information_bits_positions] = False
        self.R = self.K / self.N

    def encode(self, info_bits):
        if self.crc is not None:
            info_bits = self.crc.append_crc(info_bits)
        u = np.zeros(self.N, dtype=int)
        u[self.information_bits_positions] = info_bits[:self.K_total]
        x = self._polar_transform(u)
        return x

    def _polar_transform(self, u):
        N = len(u)
        n = int(np.log2(N))
        x = u.copy()
        for i in range(n):
            step = 2 ** i
            for j in range(0, N, 2 * step):
                for k in range(step):
                    x[j + k] ^= x[j + k + step]
        return x

    def decode(self, llr, decoder_type='SC', list_size=1):
        """
        Decodes the received LLRs using the specified decoder.
        Returns:
            decoded: Full-length estimated codeword (length N)
            decoded_info: Extracted info bits (K or K+CRC)
        """
        if decoder_type == 'SC':
            decoded = self.sc_decode(llr)
        elif decoder_type == 'SCL':
            scl_decoder = SCLDecoder(self.N, self.K_total, self.frozen_bits, list_size)
            decoded = scl_decoder.decode(llr)
        else:
            raise ValueError(f"Unsupported decoder type: {decoder_type}")

        # Extract info bits
        if self.crc:
            decoded_info = decoded[self.information_bits_positions[:self.K + self.crc.length]]
        else:
            decoded_info = decoded[self.information_bits_positions[:self.K]]

        return decoded, decoded_info


def evaluate_decoder(decoder, snr_db, num_frames, K, N, compute_mi=False):
    total_bit_errors = 0
    total_frame_errors = 0
    total_bits = 0
    total_frames = 0

    mi_llrs = None
    mi_bits = None

    for _ in range(num_frames):
        # --- Generate data ---
        info_bits = np.random.randint(0, 2, K)
        codeword = polar.encode(info_bits)
        tx = 1 - 2 * codeword  # BPSK: 0 -> +1, 1 -> -1

        # --- Add AWGN noise ---
        snr_linear = 10 ** (snr_db / 10)
        noise_std = np.sqrt(1 / (2 * snr_linear))
        noise = np.random.normal(0, noise_std, size=N)
        rx = tx + noise

        # --- Calculate LLRs ---
        llr = 2 * rx / (noise_std ** 2)

        # --- Decode ---
        decoded_bits = decoder.decode(llr)

        # --- Count errors ---
        bit_errors = np.sum(info_bits != decoded_bits)
        frame_error = int(bit_errors > 0)

        total_bit_errors += bit_errors
        total_frame_errors += frame_error
        total_bits += K
        total_frames += 1

        # Save for MI (first frame only is enough)
        if compute_mi and mi_llrs is None:
            mi_llrs = llr
            mi_bits = info_bits

    ber = total_bit_errors / total_bits
    bler = total_frame_errors / total_frames

    if compute_mi:
        return ber, bler, mi_llrs, mi_bits
    else:
        return ber, bler

    def sc_decode(self, llr):
        N = self.N
        u_hat = np.zeros(N, dtype=int)

        def recursive_sc_decode(llr, depth=0):
            n = len(llr)
            if n == 1:
                return np.array([0 if self.frozen_bits[depth] else int(llr[0] < 0)], dtype=int)

            llr_left = np.sign(llr[:n//2]) * np.minimum(np.abs(llr[:n//2]), np.abs(llr[n//2:]))

            u_hat_left = recursive_sc_decode(llr_left, depth=depth*2)
            llr_right = llr[n//2:] + ((-1) ** u_hat_left) * llr[:n//2]

            u_right = recursive_sc_decode(llr_right, depth=depth*2+1)

            u_hat[depth*2 : depth*2 + n//2] = u_hat_left
            u_hat[depth*2 + n//2 : depth*2 + n] = u_right

            return np.concatenate([u_hat_left ^ u_right, u_right])

        u_hat = recursive_sc_decode(llr)
        return u_hat


# ===========================
# Tal-Vardy SCL Decoder (list decoder)
# ===========================
class SCLDecoder:
    def __init__(self, N, K_total, frozen_bits, list_size=8):
        self.N = N
        self.K_total = K_total
        self.frozen_bits = frozen_bits
        self.list_size = list_size

    def decode(self, llr):
        paths = [([], 0.0)]  # list of (path, path_metric)

        for i in range(self.N):
            new_paths = []
            for path, metric in paths:
                if self.frozen_bits[i]:
                    bit = 0
                    new_metric = metric
                    new_paths.append((path + [bit], new_metric))
                else:
                    for bit in [0, 1]:
                        cost = np.log1p(np.exp(-llr[i] * (1 - 2 * bit)))
                        new_metric = metric + cost
                        new_paths.append((path + [bit], new_metric))

            paths = sorted(new_paths, key=lambda x: x[1])[:self.list_size]

        best_path_bits = paths[0][0]
        return np.array(best_path_bits, dtype=int)

#####################################################################################
#Main RNN
def train_rnn(model, polar, crc, device, epochs, batch_size, learning_rate, num_train_frames=10000):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.BCELoss()

    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for _ in range(num_train_frames // batch_size):
            inputs = []
            targets = []

            for _ in range(batch_size):
                info_bits = np.random.randint(0, 2, polar.K)
                full_bits = crc.append_crc(info_bits) if crc else info_bits  # length = K + CRC_LEN

                codeword = polar.encode(info_bits)
                symbols = bpsk_modulation(codeword)

                noise = np.random.randn(polar.N) * np.sqrt(1 / (2 * 10**(1.0 / 10)))  # 1 dB fixed noise
                y = symbols + noise

                llr = calculate_llr(y, np.sqrt(1 / (2 * 10**(1.0 / 10))))
                inputs.append(llr)
                targets.append(full_bits)  # Fix applied here

            inputs = torch.tensor(inputs, dtype=torch.float32).unsqueeze(2).to(device)  # (batch, N, 1)
            targets = torch.tensor(targets, dtype=torch.float32).to(device)  # (batch, K+CRC)

            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs[:, :targets.shape[1]]  # truncate to match label length
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_train_loss = epoch_loss / (num_train_frames // batch_size)
        train_losses.append(avg_train_loss)

        # Validation (optional dummy val using same method)
        model.eval()
        with torch.no_grad():
            val_inputs = []
            val_targets = []
            for _ in range(batch_size):
                info_bits = np.random.randint(0, 2, polar.K)
                full_bits = crc.append_crc(info_bits) if crc else info_bits
                codeword = polar.encode(info_bits)
                symbols = bpsk_modulation(codeword)
                noise = np.random.randn(polar.N) * np.sqrt(1 / (2 * 10**(1.0 / 10)))
                y = symbols + noise
                llr = calculate_llr(y, np.sqrt(1 / (2 * 10**(1.0 / 10))))
                val_inputs.append(llr)
                val_targets.append(full_bits)

            val_inputs = torch.tensor(val_inputs, dtype=torch.float32).unsqueeze(2).to(device)
            val_targets = torch.tensor(val_targets, dtype=torch.float32).to(device)
            val_outputs = model(val_inputs)
            val_outputs = val_outputs[:, :val_targets.shape[1]]
            val_loss = criterion(val_outputs, val_targets).item()

            val_losses.append(val_loss)

        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f}")

    return train_losses, val_losses
####################################################################################


#===========================
# Dummy Multi-layer GRU RNN Decoder
# ===========================
class DummyRNNDecoder(nn.Module):
    def __init__(self, seq_len, hidden_size=128, num_layers=2):
        super(DummyRNNDecoder, self).__init__()
        self.gru = nn.GRU(input_size=1, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.seq_len = seq_len

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out)
        out = self.sigmoid(out)
        return out.squeeze(2)  # shape: (batch, seq_len)


def rnn_decode(model, llr):
    model.eval()
    with torch.no_grad():
        llr_tensor = torch.tensor(llr, dtype=torch.float32).unsqueeze(0).unsqueeze(2)
        output = model(llr_tensor).squeeze(0).cpu().numpy()
        decoded_bits = (output < 0.5).astype(int)
    return decoded_bits


# ===========================
# Helper Functions
# ===========================
def simulate_awgn(x, snr_db):
    sigma = np.sqrt(1/(2 * (10**(snr_db/10))))
    noise = sigma * np.random.randn(*x.shape)
    y = x + noise
    return y, sigma


def bpsk_modulation(u):
    return 1 - 2 * u  # 0->+1, 1->-1


def calculate_llr(y, sigma):
    return 2 * y / (sigma ** 2)


def bit_errors(true_bits, decoded_bits, info_positions):
    decoded_info_bits = decoded_bits[info_positions]
    return np.sum(true_bits != decoded_info_bits[:len(true_bits)])


def block_error(true_bits, decoded_bits, info_positions):
    decoded_info_bits = decoded_bits[info_positions]
    return int(np.any(true_bits != decoded_info_bits[:len(true_bits)]))


def compute_mutual_information(llr, bits):
    p = 1 / (1 + np.exp(-llr))
    mi = np.mean(1 - (-bits*np.log2(p + 1e-15) - (1 - bits)*np.log2(1 - p + 1e-15)))
    return mi


# ===========================
# Dummy RNN training loop producing train and val loss curves
# ===========================
def train_rnn_dummy(model, epochs, batch_size, learning_rate, device):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        train_loss = np.exp(-epoch / (epochs / 5)) + 0.01 * np.random.rand()
        val_loss = np.exp(-epoch / (epochs / 7)) + 0.02 * np.random.rand()
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    return train_losses, val_losses


# ===========================
# Latest Main Simulation Function on 06/29/25

def main():
    # --- Configuration Parameters ---
    N = 128
    K = 64
    CRC_LEN = 8
    LIST_SIZE = [1, 4, 8, 16]
    SNR_RANGE_DB = np.arange(0.5, 4.5, 0.5)
    NUM_FRAMES_PER_SNR = 1000

    BATCH_SIZE = 64
    EPOCHS = 10
    LEARNING_RATE = 1e-3

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    crc = CRC(poly=0x07, length=CRC_LEN)
    polar = PolarCodeGenerator(N, K, crc=crc)
    rnn_decoder = DummyRNNDecoder(seq_len=N).to(device)

    # Train RNN
    print("Training RNN decoder...")
    train_losses, val_losses = train_rnn(
        model=rnn_decoder,
        polar=polar,
        crc=crc,
        device=device,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        num_train_frames=5000
    )


    ber_sc, bler_sc = [], []
    ber_rnn, bler_rnn = [], []
    ber_scl = {L: [] for L in LIST_SIZE}
    bler_scl = {L: [] for L in LIST_SIZE}
    mi_list = []

    ber_list = []
    bler_list = []
    mi_list = []



    for snr_db in SNR_RANGE_DB:
        print(f"Simulating at SNR = {snr_db} dB")
        snr_linear = 10 ** (snr_db / 10)
        noise_variance = 1 / (2 * snr_linear)

        bit_err_sc = block_err_sc = 0
        bit_err_rnn = block_err_rnn = 0
        bit_err_scl = {L: 0 for L in LIST_SIZE}
        block_err_scl = {L: 0 for L in LIST_SIZE}

        mi_total = 0
        mi_frames = 0

        for frame_idx in range(NUM_FRAMES):
            # --- Generate info bits and encode ---
            info_bits = np.random.randint(0, 2, K)
            codeword = polar.encode(info_bits)

            # --- BPSK modulation ---
            tx = 1 - 2 * codeword  # BPSK: 0→+1, 1→−1

            # --- Add AWGN noise ---
            noise = np.random.normal(0, np.sqrt(noise_variance), N)
            rx = tx + noise

            # --- Compute LLRs ---
            llrs = 2 * rx / noise_variance

            # --- Decode using SC decoder ---
            decoded_sc = decode_sc(llrs)

            # --- Error counters ---
            bit_err_sc += np.sum(decoded_sc != info_bits)
            block_err_sc += int(np.any(decoded_sc != info_bits))

            # --- Mutual Information Calculation ---
            mi = compute_mutual_information(llrs, info_bits)
            mi_total += mi
            mi_frames += 1

        # --- After frame loop ---
        mi_avg = mi_total / mi_frames
        mi_list.append(mi_avg)


  #  for snr_db in SNR_RANGE_DB:
   #     print(f"Simulating at SNR = {snr_db} dB")
    #    snr_linear = 10 ** (snr_db / 10)
     #   noise_variance = 1 / (2 * snr_linear)

        bit_err_sc = block_err_sc = 0
        bit_err_rnn = block_err_rnn = 0
        bit_err_scl = {L: 0 for L in LIST_SIZE}
        block_err_scl = {L: 0 for L in LIST_SIZE}

      #  mi_total = 0
        # ADD THIS BLOCK:
      #  llrs, transmitted_bits = get_llrs_and_bits_for_mi(...)  # <- define or capture from eval
       # mi = compute_mutual_information(llrs, transmitted_bits)
        #mi_list.append(mi)


        for _ in range(NUM_FRAMES_PER_SNR):
            info_bits = np.random.randint(0, 2, K)
            encoded_bits = polar.encode(info_bits)
            tx = bpsk_modulation(encoded_bits)
            noise = np.random.randn(N) * np.sqrt(noise_variance)
            y = tx + noise
            llr = 2 * y / noise_variance

            decoded_sc, _ = polar.decode(llr, decoder_type='SC')
            bit_err_sc += bit_errors(info_bits, decoded_sc, polar.information_bits_positions)
            block_err_sc += block_error(info_bits, decoded_sc, polar.information_bits_positions)

            for L in LIST_SIZE:
                decoded_scl, _ = polar.decode(llr, decoder_type='SCL', list_size=L)
                bit_err_scl[L] += bit_errors(info_bits, decoded_scl, polar.information_bits_positions)
                block_err_scl[L] += block_error(info_bits, decoded_scl, polar.information_bits_positions)

            with torch.no_grad():
                llr_tensor = torch.tensor(llr, dtype=torch.float32).unsqueeze(0).unsqueeze(2).to(device)
                rnn_output = rnn_decoder(llr_tensor).cpu().numpy().squeeze(0)
                decoded_rnn = (rnn_output[:K] > 0.5).astype(int)
            bit_err_rnn += np.sum(info_bits != decoded_rnn)
            block_err_rnn += int(np.any(info_bits != decoded_rnn))

            mi_total += compute_mutual_information(llr, encoded_bits)

        ber_sc.append(bit_err_sc / (NUM_FRAMES_PER_SNR * K))
        bler_sc.append(block_err_sc / NUM_FRAMES_PER_SNR)

        for L in LIST_SIZE:
            ber_scl[L].append(bit_err_scl[L] / (NUM_FRAMES_PER_SNR * K))
            bler_scl[L].append(block_err_scl[L] / NUM_FRAMES_PER_SNR)

        ber_rnn.append(bit_err_rnn / (NUM_FRAMES_PER_SNR * K))
        bler_rnn.append(block_err_rnn / NUM_FRAMES_PER_SNR)

        mi_list.append(mi_total / NUM_FRAMES_PER_SNR)

#####################################################



    # --- Plot SC decoder results ---
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.semilogy(SNR_RANGE_DB, ber_sc, marker='o', label='SC')
    plt.ylim(1e-5, 1)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Bit Error Rate (BER)')
    plt.title('SC Decoder BER vs SNR')
    plt.grid(True, which='both')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.semilogy(SNR_RANGE_DB, bler_sc, marker='o', label='SC')
    plt.ylim(1e-5, 1)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Block Error Rate (BLER)')
    plt.title('SC Decoder BLER vs SNR')
    plt.grid(True, which='both')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # --- Plot SCL decoder results ---
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    for L in LIST_SIZE:
        plt.semilogy(SNR_RANGE_DB, ber_scl[L], marker='o', label=f'SCL L={L}')
    plt.ylim(1e-5, 1)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Bit Error Rate (BER)')
    plt.title('SCL Decoder BER vs SNR')
    plt.grid(True, which='both')
    plt.legend()

    plt.subplot(1, 2, 2)
    for L in LIST_SIZE:
        plt.semilogy(SNR_RANGE_DB, bler_scl[L], marker='o', label=f'SCL L={L}')
    plt.ylim(1e-5, 1)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Block Error Rate (BLER)')
    plt.title('SCL Decoder BLER vs SNR')
    plt.grid(True, which='both')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # --- Plot RNN decoder results ---
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.semilogy(SNR_RANGE_DB, ber_rnn, marker='o', color='purple', label='RNN')
    plt.ylim(1e-5, 1)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Bit Error Rate (BER)')
    plt.title('RNN Decoder BER vs SNR')
    plt.grid(True, which='both')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.semilogy(SNR_RANGE_DB, bler_rnn, marker='o', color='purple', label='RNN')
    plt.ylim(1e-5, 1)
    plt.xlabel('SNR (dB)')
    plt.ylabel('Block Error Rate (BLER)')
    plt.title('RNN Decoder BLER vs SNR')
    plt.grid(True, which='both')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # --- Plot Training and Validation Loss ---
    plt.figure()
    plt.plot(range(1, EPOCHS + 1), train_losses, label='Training Loss')
    plt.plot(range(1, EPOCHS + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('RNN Training and Validation Loss')
    plt.grid(True)
    plt.legend()
    plt.show()

    # --- Plot Mutual Information (placeholder) ---
    # --- MI Plot ---
    if len(mi_list) == len(SNR_RANGE_DB):
        plt.figure()
        plt.plot(SNR_RANGE_DB, mi_list, marker='o')
        plt.xlabel('SNR (dB)')
        plt.ylabel('Mutual Information (bits)')
        plt.title('Mutual Information vs SNR')
        plt.grid(True)
        plt.show()
    mi_list = []


if __name__ == "__main__":
    main()





Training RNN decoder...
Epoch 1/10 - Train Loss: 0.6933 | Val Loss: 0.6931
Epoch 2/10 - Train Loss: 0.6932 | Val Loss: 0.6932
Epoch 3/10 - Train Loss: 0.6932 | Val Loss: 0.6930
